In [1]:
import panel as pn
import pandas as pd
import numpy as np
# pn.extension()
pn.extension('deckgl')
import pydeck as pdk
file_input = pn.widgets.FileInput(accept='.csv', name='Upload CSV')
GREEN_RGB = [0, 255, 0, 90]
RED_RGB = [240, 100, 0, 90]

def load_csv(data):
    import io
    if data is not None:
        global df
        df = pd.read_csv(io.BytesIO(data))
        global orgin_lon, orgin_lat, dest_lon, dest_lat
        orgin_lon = pn.widgets.Select(name="Origin Lontitue", value="ZIP_lon", options=list(df.columns))
        orgin_lat = pn.widgets.Select(name="Origin Latitude", value="ZIP_lat", options=list(df.columns) )
        dest_lon = pn.widgets.Select(name="Destination Longitude", value="AHA_ID_lon", options=list(df.columns))
        dest_lat = pn.widgets.Select(name="Destination Latitude", value="AHA_ID_lat", options=list(df.columns))
        od_col_slect_view = pn.Column("## 2. Select the columns for origin and destination points", pn.Row(orgin_lon, orgin_lat, dest_lon, dest_lat))

        return pn.Column("Preview of the uploaded data (first 3 rows, there are %d rows in total)"%df.shape[0],pn.pane.DataFrame(df.head(3)),
                        #  deck_gl,
                         od_col_slect_view
                        )


active_load_csv = pn.bind(load_csv, file_input.param.value)


# pn.Column(file_input, active_load_csv)

data_upload_view = pn.Column("## 1. Select the data that need to be calculated",file_input,active_load_csv)
data_upload_view

BokehModel(combine_events=True, render_bundle={'docs_json': {'e46a29eb-9055-4bce-ae9f-b1f137e643f9': {'version…

In [2]:
from georouting.routers import OSRMRouter
from panel.widgets import Tqdm
tqdm = Tqdm()
# create a router object 
router = OSRMRouter(mode="driving",
                    base_url="http://172.30.232.152:5000"
                    )

def calculate_distance(run):
    if not run:
        yield "Calculation did not finish yet :("
        return
    # import gevent
    for k,v in tqdm(df.iterrows(), total=df.shape[0]):
        origin = ( v[orgin_lat.value],v[orgin_lon.value])
        destination = ( v[dest_lat.value],v[dest_lon.value])
        route = router.get_route(origin, destination)
        df.loc[k, 'distance (m)'] = route.get_distance()
        df.loc[k, 'duration (s)'] = route.get_duration()
    
    final_table = pn.pane.DataFrame(df.head(3), 
                                    # sizing_mode='stretch_width'
                                    )
    
    # geoview
    arc_layer = pdk.Layer(
                "ArcLayer",
                data=df,
                # get_width="S000 * 60",
                get_width="2",
                # set arc width

                get_source_position=[orgin_lon.value, orgin_lat.value],
                get_target_position=[dest_lon.value, dest_lat.value],
                get_tilt=15,
                get_source_color=RED_RGB,
                get_target_color=GREEN_RGB,
                pickable=True,
                auto_highlight=True,
            )
    import numpy as np
    v1 = df[[orgin_lon.value, orgin_lat.value]].values
    v2 = df[[dest_lon.value, dest_lat.value]].values
    v = np.concatenate([v1, v2])
    v_df = pd.DataFrame(v, columns=["lon", "lat"])
    data_view = pdk.data_utils.compute_view(v_df[["lon", "lat"]])
    view_state = pdk.ViewState(
                    longitude=data_view.longitude, latitude=data_view.latitude, zoom= data_view.zoom, bearing=0, pitch=45
                )
    TOOLTIP_TEXT = {"html": "<br /> Source location in red; Destination location in green"}
    r = pdk.Deck(arc_layer, initial_view_state=view_state, tooltip=TOOLTIP_TEXT)

    MAPBOX_KEY = "pk.eyJ1IjoicGFuZWxvcmciLCJhIjoiY2s1enA3ejhyMWhmZjNobjM1NXhtbWRrMyJ9.B_frQsAVepGIe-HiOJeqvQ"
    import json
    json_spec = json.loads(r.to_json())
    deck_gl = pn.pane.DeckGL(json_spec, mapbox_api_key=MAPBOX_KEY, 
                            #  sizing_mode='stretch_width',
                            # height=600
                            )


    from io import StringIO
    sio = StringIO()
    df.to_csv(sio)
    sio.seek(0)
    download_view = pn.widgets.FileDownload(sio, embed=True, filename='results.csv', 
                                            # sizing_mode='stretch_width'
                                            )
    results_desc = pn.pane.Markdown("""
    ## 3. Downlaod the result
    
    The arc map of the source and destination points. Source location in red, destination location in green. The map can only show 10000 rows at most.
                             """)
    table_download_view = pn.Column(results_desc,deck_gl,"Preview of the result. It only show the first 3 rows at most.",final_table,download_view)
    
    yield table_download_view


run = pn.widgets.Button(name="Press to run the calculation")

run_and_download = pn.Column(run, tqdm, pn.bind(calculate_distance, run))

         
# pn.Column(button,tqdm)
# table_download_view = pn.bind(table_download, button.param.value)
# run_and_download = pn.Column(button,tqdm,table_download_view)
run_and_download

BokehModel(combine_events=True, render_bundle={'docs_json': {'dc50e105-cc0e-46d1-a9ff-9adf72f3378c': {'version…

In [25]:
# from tqdm.asyncio import tqdm

# with tqdm(range(9)) as pbar:
#     async for i in pbar:
#         if i == 2:
            # break

In [27]:
# import gevent

# def foo():
#     print('Running in foo')
#     gevent.sleep(0)
#     print('Explicit context switch to foo again')

# def bar():
#     print('Explicit context to bar')
#     gevent.sleep(0)
#     print('Implicit context switch back to bar')

# temp = gevent.joinall([
#     gevent.spawn(foo),
#     gevent.spawn(bar),
# ])

In [30]:
intro = pn.pane.Markdown("""
 
This app calculates distance and duration between two points using a `CSV` input with longitude and latitude columns for both start and end points. See the sample data below or download the [sample CSV](https://raw.githubusercontent.com/spatial-data-lab/data/main/sample_3.csv) ( the [sample CSV](https://raw.githubusercontent.com/spatial-data-lab/data/main/sample_3.csv) includes comparison columns like the results from Google, which are not required). Ensure you select the right columns for origin and destination in the app. The output is a CSV with added `distance (m)` and `duration (s)` columns.

| AHA_ID_lon         | AHA_ID_lat  | ZIP_lon            | ZIP_lat      |
|--------------------|-------------|--------------------|--------------|
| -73.91651806999998 | 42.81997773 | -72.605400453      | 42.376239033 |
| -72.68278784       | 41.75384063 | -72.967189321      | 42.293923206 |

""", 
sizing_mode='stretch_width'
)
intro

BokehModel(combine_events=True, render_bundle={'docs_json': {'8cf5aaf2-5ab1-4095-a17f-10081457411f': {'version…

In [ ]:
contribute = pn.pane.Markdown("""
Please cite [our paper](https://isprs-archives.copernicus.org/articles/XLVIII-4-W7-2023/53/2023/) if you use this app for your research. This app is built with [OSRM](http://project-osrm.org/), [Panel](https://panel.holoviz.org/), [Georouting](https://github.com/wybert/georouting) and [Pydeck.gl](https://pydeck.gl/). It hosted in [New England Research Cloud (NERC)](https://nerc.mghpcc.org/). The road network data is from [OpenStreetMap](https://www.openstreetmap.org/). We use Multi-Level Dijkstra (MLD) algorithm to find the route. For comparison with other routing engines, like Google Maps, Bing Maps, ESRI Routing service etc., for more detail please check [our paper](https://isprs-archives.copernicus.org/articles/XLVIII-4-W7-2023/53/2023/) on FOSS4G 2023. 

This app is developed by [Xiaokang Fu](https://gis.harvard.edu/people/xiaokang-fu) and [Devika Kakkar](https://gis.harvard.edu/people/devika-kakkar). Please contact [Devika Kakkar](mailto:kakkar@fas.harvard.edu) for any questions. 

""", 
sizing_mode='stretch_width'
)
contribute

In [5]:

app = pn.Column(
       intro,
        data_upload_view,
        run_and_download,
        contribute
        )
app

BokehModel(combine_events=True, render_bundle={'docs_json': {'8a8bd16c-019a-4fc7-8920-e226e9e36997': {'version…

In [6]:
# app.show()

In [7]:
# Instantiate the template with widgets displayed in the sidebar
template = pn.template.FastListTemplate(
    title='Rapid Route',
    logo='https://dssg.fas.harvard.edu/wp-content/uploads/2017/12/CGA_logo_globe_400x400.jpg',
    favicon = 'https://dssg.fas.harvard.edu/wp-content/uploads/2017/12/CGA_logo_globe_400x400.jpg',
    header_background = '#212121',
    header_color = '#2F6DAA',
)

# set relative sizing of the main area and sidebar

# Append a layout to the main area, to demonstrate the list-like API
template.main.append(app)
template.main.append(app)
template.servable()

FastListTemplate
    [js_area] HTML(None, height=0, margin=0, sizing_mode='fixed', width=0)
    [actions] TemplateActions()
    [browser_info] BrowserInfo()
    [busy_indicator] LoadingSpinner(height=20, width=20)
    [10751445072] Column
        [0] Markdown(str, sizing_mode='stretch_width')
        [1] Column
            [0] Markdown(str)
            [1] FileInput(accept='.csv', name='Upload CSV')
            [2] ParamFunction(function, _pane=Str, defer_load=False)
        [2] Column
            [0] Button(name='Press to run c...)
            [1] Tqdm(layout=Column, progress=Progress, text_pane=Str)
            [2] ParamFunction(function, _pane=Markdown, defer_load=False)